In [25]:
# load .env file
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
load_dotenv()


True

In [26]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
# from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

In [27]:
model = ChatOpenAI(model="gpt-3.5-turbo")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGSMITH_PAT")

In [28]:
embedding_model =  HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [30]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://www.techmango.net/",),
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [34]:
print(rag_chain.invoke("Techmango services"))

I don't know.
